In [2]:
import warnings
warnings.filterwarnings('ignore')
import keras
import glob
from tqdm import *
import os
import cv2
import numpy as np
from keras.preprocessing.image import load_img, img_to_array


In [3]:
num_classes = 42
EPOCHS = 15
BS = 50

In [ ]:
train_dirs = glob.glob("vision_final_prj/processed_dataset/*")
train_dirs.sort()
train_dirs = train_dirs[:num_classes]

data = []
labels = []
for train_dir in tqdm(train_dirs):
  imgPaths = glob.glob(train_dir + "/*.jpg")
  imgPaths.sort()
  for imgPath in tqdm(imgPaths):
    image = load_img(imgPath, target_size=(64, 64))
    image = img_to_array(image) 
    data.append(image)

    label = imgPath.split(os.path.sep)[-2]
    label = int(label)
    labels.append(label)

 24%|██▍       | 497/2086 [00:06<00:35, 44.25it/s]

In [ ]:
print(len(data), len(labels))

In [ ]:
data = np.array(data, dtype=np.float) / 255.
labels = np.array(labels)

In [ ]:
from sklearn.model_selection import train_test_split
train_input, valid_input, train_target, valid_target = train_test_split(data,
                                                                        labels,
                                                                        test_size=0.25,
                                                                        random_state=123)

In [ ]:
from keras.utils import to_categorical
train_target = to_categorical(train_target, num_classes=num_classes)
valid_target = to_categorical(valid_target, num_classes=num_classes)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
aug = ImageDataGenerator(rotation_range=10, width_shift_range=0.1, height_shift_range=0.1)

In [9]:
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense , BatchNormalization ,Activation
from keras.models import Model
from keras.applications import VGG16
#from keras.preprocessing import image

IMAGE_SIZE = [64,64]  # we will keep the image size as (64,64). You can increase the size for better results. 

# loading the weights of VGG16 without the top layer. These weights are trained on Imagenet dataset.
vgg = VGG16(input_shape = IMAGE_SIZE + [3], weights = 'imagenet', include_top = False)  # input_shape = (64,64,3) as required by VGG

# this will exclude the initial layers from training phase as there are already been trained.
for layer in vgg.layers:
    layer.trainable = False

x = Flatten()(vgg.output)
#x = Dense(128, activation = 'relu')(x)   # we can add a new fully connected layer but it will increase the execution time.
x = Dense(num_classes, activation = 'softmax')(x)  # adding the output layer with softmax function as this is a multi label classification problem.

model = Model(inputs = vgg.input, outputs = x)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

ValueError: Input size must be at least 32x32; got `input_shape=[28, 28, 3]`